# Creating a Reference table from Ensembl

This notebook takes a user list of gene names and returns a table 
from Ensembl with variants in the exons of the given genes

In [1]:
import requests, sys
import pandas as pd
import json
import pprint
import ast
import time
import ast

## 1. Import a list of genes, look up their ENSG, ENST and ENSE identifiers

In [3]:
#list of genes

genefile = open('input/big_MD_gene_list.txt', 'r')
genelist = []
for line in genefile:
    genelist.append(line.rstrip("\n"))
len(genelist) #how many genes are in the list

101

Look up identifiers and exons from gene name using Ensembl API requesting the https://rest.ensembl.org/ server. 

In [4]:
passed_genes_list = []
json_list = []

In [5]:
server = "https://rest.ensembl.org"
for gene in genelist:
    if gene not in passed_genes_list:
        ext = ("/lookup/symbol/homo_sapiens/" + gene + "?expand=1")
        r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
        if not r.ok:
            print('unsuccesful ', gene)
            continue
 
        decoded = r.json()
        gene_ids = json.loads(repr(decoded).replace("\'", "\""))
        json_list.append(gene_ids)
        time.sleep(1)
        passed_genes_list.append(gene)
        print(gene)
    else: continue

APPL1
CSMD1
CD8B
PPP1R3A
GLUD1
LPL
GNG3
SLC2A2
GLIS3
WFS1
ITGB3
RMND5A
STX11
unsuccesful  SNORA98
MAGEL2
HNF1A
PPP1R15B
PTF1A
LEP
FOXP3
PHACTR2
PARK7
CIDEC
GPR161
CD8A
SHANK3
GCK
SF3B5
MC4R
FBN1
PURA
KCNQ1
AGPS
RFX6
NEUROG3
NKX6-1
LMNA
HBB
GRIN2B
SLC19A2
TRMT10A
MBL2
PAX4
LTV1
PLIN1
SHLD2
IER3IP1
C12orf43
CEL
STAT3
UTRN
PPARG
KCNH2
INS
PRKAG2
HNF4A
CAVIN1
TH
BLK
GATA4
MLKL
HADH
IGF2
TRIP11
ZFP57
PAX6
SIM1
EIF2AK3
EFL1
AGPAT2
AKT2
PDX1
LZTR1
RET
NEUROD1
KCNJ11
HNF1B
INSR
HYMAI
PLAGL1
PCBD1
MECP2
LEPR
ZMPSTE24
GATA6
PDIA6
ALMS1
EDEM2
LAMA2
KMT2E
ABCC8
BSCL2
ZC2HC1B
NLRP3
FOXP1
CAV1
ASB14
KCNQ2
SCN1A
KLF11
PAX2


In [6]:
#here is our list of jsons, each contains one gene

json_list
pprint.pprint(json_list)

[{'Transcript': [{'Exon': [{'assembly_name': 'GRCh38',
                            'db_type': 'core',
                            'end': 57227937,
                            'id': 'ENSE00003837686',
                            'object_type': 'Exon',
                            'seq_region_name': '3',
                            'species': 'homo_sapiens',
                            'start': 57227726,
                            'strand': 1,
                            'version': 1},
                           {'assembly_name': 'GRCh38',
                            'db_type': 'core',
                            'end': 57235664,
                            'id': 'ENSE00003577757',
                            'object_type': 'Exon',
                            'seq_region_name': '3',
                            'species': 'homo_sapiens',
                            'start': 57235566,
                            'strand': 1,
                            'version': 1},
                      

                 {'Exon': [{'assembly_name': 'GRCh38',
                            'db_type': 'core',
                            'end': 120979094,
                            'id': 'ENSE00003536103',
                            'object_type': 'Exon',
                            'seq_region_name': '12',
                            'species': 'homo_sapiens',
                            'start': 120978626,
                            'strand': 1,
                            'version': 1},
                           {'assembly_name': 'GRCh38',
                            'db_type': 'core',
                            'end': 120989032,
                            'id': 'ENSE00003700642',
                            'object_type': 'Exon',
                            'seq_region_name': '12',
                            'species': 'homo_sapiens',
                            'start': 120988833,
                            'strand': 1,
                            'version': 1},
                

                 {'Exon': [{'assembly_name': 'GRCh38',
                            'db_type': 'core',
                            'end': 177393124,
                            'id': 'ENSE00003910487',
                            'object_type': 'Exon',
                            'seq_region_name': '2',
                            'species': 'homo_sapiens',
                            'start': 177393036,
                            'strand': 1,
                            'version': 1},
                           {'assembly_name': 'GRCh38',
                            'db_type': 'core',
                            'end': 177395678,
                            'id': 'ENSE00003911513',
                            'object_type': 'Exon',
                            'seq_region_name': '2',
                            'species': 'homo_sapiens',
                            'start': 177395589,
                            'strand': 1,
                            'version': 1},
                  

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
#writing the resulting list in the file just in case the connection is lost

with open('gene_id_decoded_json_list.txt', 'w') as output:
    output.write(str(json_list))

In [8]:
#extracting exon ids from it keeping the gene ids

columns = ['Gene', 'Transcript', 'Exon']
df_data = []

for gene in json_list:
    for elm in gene['Transcript']:
        geneID = elm['Parent']
        transcriptID = elm['id']
        for exon in elm['Exon']:
            df_data.append([geneID, transcriptID, exon['id']])
            
df = pd.DataFrame(data=df_data, columns=columns)
df

,Gene,Transcript,Exon
0,ENSG00000157500,ENST00000650354,ENSE00003837686
1,ENSG00000157500,ENST00000650354,ENSE00003577757
2,ENSG00000157500,ENST00000650354,ENSE00003599120
3,ENSG00000157500,ENST00000650354,ENSE00003522948
4,ENSG00000157500,ENST00000650354,ENSE00003560629
...,...,...,...
14009,ENSG00000075891,ENST00000554172,ENSE00003607033
14010,ENSG00000075891,ENST00000554172,ENSE00002435858
14011,ENSG00000075891,ENST00000554172,ENSE00003458791
14012,ENSG00000075891,ENST00000554172,ENSE00000721143


In [9]:
#saving the Gene, Transcript and Exon IDs to the file

df.to_csv('gene_trans_exon.csv', header=True, index=False)

## 2. Calling for all the variants in these regions (exons of the selected genes) existing on Ensembl

In [10]:
passed_exons_list = []
exon_variants_list = []

In [11]:
server = "https://rest.ensembl.org"

for index,row in df.iterrows(): 
    if row['Exon'] not in passed_exons_list:
        ext_exon = ("/overlap/id/" + row['Exon'] + "?feature=variation")
        geneID = row['Gene']
        transcriptID = row['Transcript']
        exonID = row['Exon']

        resp = requests.get(server+ext_exon, headers={ "Content-Type" : "application/json"})  
        if not resp.ok:
            print('failed for ' + str(index))
            continue
        var_decoded = resp.json()
        variants = pd.DataFrame(json.loads(repr(var_decoded).replace("\'", "\"")))
        variants['Gene'] = geneID #add the column with Gene ID to have it in the final df
        variants['Transcript'] = transcriptID #same for Transcript ID
        variants['Exon'] = exonID #same for Exon ID
        exon_variants_list.append(variants)
        time.sleep(1)
        print(index)
        passed_exons_list.append(row['Exon'])
    else: continue

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
71
74
75
76
82
88
89
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
199
214
219
228
288
357
358
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
549
550
554
556
557
558
559
560
562
563
564
565
566
567
568
569
570
571
574
575
576
577
582
588
589
594
600
601
602
603
604
605
606
607
608
609
610
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
646
647
651
652
653
654
655
666
667
668
670
683
684
686
699
700
701
714
721
722
731
742
743
755
756
767
768
769
773
774
784
785
795
808

4591
4592
4593
4601
4602
4611
4612
4613
4621
4622
4629
4632
4639
4642
4650
4652
4658
4659
4661
4662
4663
4664
4665
4666
4674
4682
4684
4685
4687
4688
4689
4692
4693
4696
4697
4702
4703
4708
4709
4714
4715
4720
4721
4722
4727
4728
4729
4735
4738
4741
4742
4747
4748
4749
4754
4757
4758
4759
4760
4761
4762
4763
4764
4765
4766
4767
4768
4769
4770
4771
4772
4773
4774
4775
4776
4777
4778
4779
4780
4781
4782
4783
4784
4785
4786
4787
4788
4789
4790
4791
4802
4803
4804
4810
4811
4813
4814
4815
4819
4820
4821
4822
4823
4824
4825
4826
4827
4829
4832
4833
4834
4835
4836
4837
4838
4839
4840
4841
4842
4843
4844
4845
4846
4847
4848
4849
4850
4851
4852
4863
4864
4867
4877
4878
4879
4883
4884
4885
4886
4887
4888
4889
4890
4891
4892
4893
4894
4895
4910
4911
4922
4923
4924
4935
4936
4937
4938
4939
4940
4947
4949
4950
4951
4952
4957
4960
4966
4971
4972
4987
4988
4989
4997
4998
5001
5002
5015
5016
5018
5028
5033
5042
5043
5058
5059
5066
5067
5071
5072
5074
5075
5090
5091
5092
5093
5098
5101
5102
5106
5111


8640
8641
8642
8643
8644
8645
8646
8647
8648
8649
8650
8651
8652
8653
8654
8655
8656
8657
8658
8659
8660
8661
8662
8663
8664
8665
8666
8667
8668
8669
8670
8671
8672
8673
8674
8675
8676
8677
8678
8679
8680
8681
8682
8683
8684
8685
8686
8687
8688
8689
8690
8691
8692
8693
8694
8695
8696
8697
8698
8699
8700
8701
8754
8766
8787
8833
8834
8835
8836
8837
8838
8839
8840
8841
8842
8843
8844
8845
8853
8854
8855
8856
8857
8858
8862
8863
8871
8872
8873
8878
8879
8880
8881
8882
8883
8884
8885
8886
8887
8888
8889
8890
8891
8892
8893
8894
8895
8896
8897
8898
8902
8903
8904
8905
8906
8907
8908
8909
8910
8911
8912
8913
8914
8915
8924
8933
8934
8944
8950
8951
8952
8959
8960
8961
8962
8963
8964
8965
8966
8967
8969
8970
8971
8972
8973
8974
8975
8976
8977
8978
8979
8980
8982
8983
8984
8993
8994
8995
8996
8997
8998
8999
9000
9001
9002
9003
9004
9005
9006
9007
9008
9009
9010
9011
9024
9025
9026
9027
9028
9044
9045
9046
9047
9053
9054
9055
9056
9057
9058
9059
9070
9088
9089
9090
9091
9092
9093
9094
9095
9120


In [16]:
#writing the resulting list in the file just in case the connection is lost

with open('exon_variants_list.txt', 'w') as output:
    output.write(str(exon_variants_list))

In [17]:
#creating the dataframe of variants

exon_variants_all_genes = pd.concat(exon_variants_list, ignore_index=True)
exon_variants_all_genes

,feature_type,consequence_type,clinical_significance,start,end,seq_region_name,assembly_name,alleles,strand,id,source,Gene,Transcript,Exon
0,variation,5_prime_UTR_variant,[],57227726.0,57227726.0,3,GRCh38,"[C, G, T]",1.0,rs1007095765,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686
1,variation,5_prime_UTR_variant,[],57227729.0,57227729.0,3,GRCh38,"[G, A, T]",1.0,rs908576182,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686
2,variation,5_prime_UTR_variant,[],57227731.0,57227731.0,3,GRCh38,"[A, C, G]",1.0,rs578118925,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686
3,variation,5_prime_UTR_variant,[],57227732.0,57227732.0,3,GRCh38,"[C, A, T]",1.0,rs965683268,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686
4,variation,5_prime_UTR_variant,[likely benign],57227733.0,57227733.0,3,GRCh38,"[G, C, T]",1.0,rs113307246,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583101,variation,missense_variant,[],100781336.0,100781336.0,10,GRCh38,"[C, T]",1.0,rs1234454137,dbSNP,ENSG00000075891,ENST00000554172,ENSE00002435858
583102,variation,missense_variant,[],100781339.0,100781339.0,10,GRCh38,"[A, G]",1.0,rs1257925666,dbSNP,ENSG00000075891,ENST00000554172,ENSE00002435858
583103,variation,missense_variant,[],100781340.0,100781340.0,10,GRCh38,"[T, A, C]",1.0,rs758231319,dbSNP,ENSG00000075891,ENST00000554172,ENSE00002435858
583104,variation,missense_variant,[],100824650.0,100824650.0,10,GRCh38,"[C, T]",1.0,rs761402395,dbSNP,ENSG00000075891,ENST00000554172,ENSE00003739908


In [89]:
#saving the unfiltered table to the file

#exon_variants_all_genes.to_csv(
#    '/Users/ksenia/Documents/MODY_genes/whole_pipeline_311011/MD_genes_exon_variants_Ens_unfiltered.csv',
#    header=True, index=False)

## 3. Filter the variants by consequence type

In [18]:
#Drop all the rows with no allele info

dbSNP = exon_variants_all_genes[
    exon_variants_all_genes['alleles'].apply(lambda x: 'HGMD_MUTATION' not in x)].reset_index(drop=True)
dbSNP

,feature_type,consequence_type,clinical_significance,start,end,seq_region_name,assembly_name,alleles,strand,id,source,Gene,Transcript,Exon
0,variation,5_prime_UTR_variant,[],57227726.0,57227726.0,3,GRCh38,"[C, G, T]",1.0,rs1007095765,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686
1,variation,5_prime_UTR_variant,[],57227729.0,57227729.0,3,GRCh38,"[G, A, T]",1.0,rs908576182,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686
2,variation,5_prime_UTR_variant,[],57227731.0,57227731.0,3,GRCh38,"[A, C, G]",1.0,rs578118925,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686
3,variation,5_prime_UTR_variant,[],57227732.0,57227732.0,3,GRCh38,"[C, A, T]",1.0,rs965683268,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686
4,variation,5_prime_UTR_variant,[likely benign],57227733.0,57227733.0,3,GRCh38,"[G, C, T]",1.0,rs113307246,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540235,variation,missense_variant,[],100781336.0,100781336.0,10,GRCh38,"[C, T]",1.0,rs1234454137,dbSNP,ENSG00000075891,ENST00000554172,ENSE00002435858
540236,variation,missense_variant,[],100781339.0,100781339.0,10,GRCh38,"[A, G]",1.0,rs1257925666,dbSNP,ENSG00000075891,ENST00000554172,ENSE00002435858
540237,variation,missense_variant,[],100781340.0,100781340.0,10,GRCh38,"[T, A, C]",1.0,rs758231319,dbSNP,ENSG00000075891,ENST00000554172,ENSE00002435858
540238,variation,missense_variant,[],100824650.0,100824650.0,10,GRCh38,"[C, T]",1.0,rs761402395,dbSNP,ENSG00000075891,ENST00000554172,ENSE00003739908


In [19]:
#saving the unfiltered table that contains variants with known allele information

dbSNP.to_csv(
    'MD_genes_exon_variants_Ens_unfiltered_dbSNP.csv',
    header=True, index=False)

Here another notebook '2_Pathogenicity_consequences_type_analysis_Ensembl' has to be run to decide for the consequence types to leave in the table

In [20]:
dbSNP['consequence_type'].unique()

array(['5_prime_UTR_variant', 'start_lost', 'missense_variant',
       'frameshift_variant', 'synonymous_variant', 'stop_gained',
       'splice_region_variant', 'inframe_deletion',
       'splice_donor_variant', 'splice_donor_5th_base_variant',
       'protein_altering_variant', 'splice_polypyrimidine_tract_variant',
       'stop_lost', '3_prime_UTR_variant', 'splice_acceptor_variant',
       'splice_donor_region_variant',
       'non_coding_transcript_exon_variant', 'coding_sequence_variant',
       'inframe_insertion', 'stop_retained_variant', 'intron_variant',
       'incomplete_terminal_codon_variant', 'mature_miRNA_variant'],
      dtype=object)

In [21]:
#The list of consequence types we decided to leave in the table

consequence_type_list = ['missense_variant',
                         'protein_altering_variant',
                         'coding_sequence_variant',
                         'frameshift_variant',
                         'splice_donor_variant',
                         'splice_acceptor_variant',
                         'splice_donor_5th_base_variant',
                         'start_lost',
                         'stop_gained',
                         'stop_lost',
                         'inframe_deletion',
                         'inframe_insertion']

In [22]:
filtered_cons_type = dbSNP.query('consequence_type in @consequence_type_list').reset_index(drop=True)
filtered_cons_type

,feature_type,consequence_type,clinical_significance,start,end,seq_region_name,assembly_name,alleles,strand,id,source,Gene,Transcript,Exon
0,variation,start_lost,[],57227885.0,57227885.0,3,GRCh38,"[T, A]",1.0,rs1196762591,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686
1,variation,missense_variant,[],57227888.0,57227888.0,3,GRCh38,"[C, A, G]",1.0,rs752732231,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686
2,variation,frameshift_variant,[],57227889.0,57227892.0,3,GRCh38,"[GGGG, GGG]",1.0,rs1559504574,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686
3,variation,missense_variant,[],57227890.0,57227890.0,3,GRCh38,"[G, A, C]",1.0,rs1022835366,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686
4,variation,missense_variant,[],57227891.0,57227891.0,3,GRCh38,"[G, A]",1.0,rs1246196168,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199122,variation,missense_variant,[],100781336.0,100781336.0,10,GRCh38,"[C, T]",1.0,rs1234454137,dbSNP,ENSG00000075891,ENST00000554172,ENSE00002435858
199123,variation,missense_variant,[],100781339.0,100781339.0,10,GRCh38,"[A, G]",1.0,rs1257925666,dbSNP,ENSG00000075891,ENST00000554172,ENSE00002435858
199124,variation,missense_variant,[],100781340.0,100781340.0,10,GRCh38,"[T, A, C]",1.0,rs758231319,dbSNP,ENSG00000075891,ENST00000554172,ENSE00002435858
199125,variation,missense_variant,[],100824650.0,100824650.0,10,GRCh38,"[C, T]",1.0,rs761402395,dbSNP,ENSG00000075891,ENST00000554172,ENSE00003739908


In [24]:
filtered_cons_type.to_csv(
    'MD_genes_exon_variants_Ens_filtered.csv',
    header=True, index=False)

## 4. Crealing a "Location" column for future mapping

To create a coordinate for each alternative allele in case there are more then 2 alleles, add a line for each alternative allele

In [28]:
#for each variant that has more than 2 alleles, add the row with each allele
alternative_alleles = []
for index, row in filtered_cons_type.iterrows():
    if len(row['alleles']) > 2:
        for i in range(len(row['alleles'])):
            if row['alleles'][0] != row['alleles'][i-1]:
                alternative_alleles.append({'clinical_significance': '',
                                            'seq_region_name': row['seq_region_name'],
                                            'assembly_name': row['assembly_name'],
                                            'alleles': [row['alleles'][0], row['alleles'][i-1]],
                                            'id': row['id'],
                                            'strand': row['strand'],
                                            'consequence_type': '',
                                            'feature_type': row['feature_type'],
                                            'source': row['source'],
                                            'end': row['end'],
                                            'start': row['start'],
                                            'Gene': row['Gene'],
                                            'Transcript': row['Transcript'],
                                            'Exon': row['Exon']})
alternative_alleles_df = pd.DataFrame(alternative_alleles)

In [29]:
all_alleles = pd.concat([filtered_cons_type, alternative_alleles_df]).reset_index(drop=True)
all_alleles

,feature_type,consequence_type,clinical_significance,start,end,seq_region_name,assembly_name,alleles,strand,id,source,Gene,Transcript,Exon
0,variation,start_lost,[],57227885.0,57227885.0,3,GRCh38,"[T, A]",1.0,rs1196762591,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686
1,variation,missense_variant,[],57227888.0,57227888.0,3,GRCh38,"[C, A, G]",1.0,rs752732231,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686
2,variation,frameshift_variant,[],57227889.0,57227892.0,3,GRCh38,"[GGGG, GGG]",1.0,rs1559504574,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686
3,variation,missense_variant,[],57227890.0,57227890.0,3,GRCh38,"[G, A, C]",1.0,rs1022835366,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686
4,variation,missense_variant,[],57227891.0,57227891.0,3,GRCh38,"[G, A]",1.0,rs1246196168,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270659,variation,,,100781275.0,100781275.0,10,GRCh38,"[A, C]",1.0,rs200741999,dbSNP,ENSG00000075891,ENST00000554172,ENSE00002435858
270660,variation,,,100781276.0,100781276.0,10,GRCh38,"[G, T]",1.0,rs201925042,dbSNP,ENSG00000075891,ENST00000554172,ENSE00002435858
270661,variation,,,100781276.0,100781276.0,10,GRCh38,"[G, C]",1.0,rs201925042,dbSNP,ENSG00000075891,ENST00000554172,ENSE00002435858
270662,variation,,,100781340.0,100781340.0,10,GRCh38,"[T, C]",1.0,rs758231319,dbSNP,ENSG00000075891,ENST00000554172,ENSE00002435858


Open all_alleles from the file if it has already been created

In [30]:
vf_allele =[]
for el in all_alleles['alleles']:
    vf_allele.append(el[1])
all_alleles['vf_allele'] = vf_allele

Here one needs to keep in mind that allelic info for the variants with more than 2 alleles stayed intact and more lines with alternativa alleles were added. In other words, one need to reguard only [o] and [1] alleles from the 'alleles' column

Correcting the ['alleles'] column so there are just 2 alleles as the ones that are more than 2 were adeed to the end of the table 

In [31]:
two_alleles = []
for item in all_alleles['alleles']:
    two_alleles.append(item[:2]) 
all_alleles['alleles'] = two_alleles
all_alleles

,feature_type,consequence_type,clinical_significance,start,end,seq_region_name,assembly_name,alleles,strand,id,source,Gene,Transcript,Exon,vf_allele
0,variation,start_lost,[],57227885.0,57227885.0,3,GRCh38,"[T, A]",1.0,rs1196762591,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686,A
1,variation,missense_variant,[],57227888.0,57227888.0,3,GRCh38,"[C, A]",1.0,rs752732231,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686,A
2,variation,frameshift_variant,[],57227889.0,57227892.0,3,GRCh38,"[GGGG, GGG]",1.0,rs1559504574,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686,GGG
3,variation,missense_variant,[],57227890.0,57227890.0,3,GRCh38,"[G, A]",1.0,rs1022835366,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686,A
4,variation,missense_variant,[],57227891.0,57227891.0,3,GRCh38,"[G, A]",1.0,rs1246196168,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270659,variation,,,100781275.0,100781275.0,10,GRCh38,"[A, C]",1.0,rs200741999,dbSNP,ENSG00000075891,ENST00000554172,ENSE00002435858,C
270660,variation,,,100781276.0,100781276.0,10,GRCh38,"[G, T]",1.0,rs201925042,dbSNP,ENSG00000075891,ENST00000554172,ENSE00002435858,T
270661,variation,,,100781276.0,100781276.0,10,GRCh38,"[G, C]",1.0,rs201925042,dbSNP,ENSG00000075891,ENST00000554172,ENSE00002435858,C
270662,variation,,,100781340.0,100781340.0,10,GRCh38,"[T, C]",1.0,rs758231319,dbSNP,ENSG00000075891,ENST00000554172,ENSE00002435858,C


In [32]:
#creating the Location column

location = []
for index, row in all_alleles.iterrows():
    location.append(str(row['seq_region_name']) + ':' + str(int(row['start'])))

In [36]:
all_alleles['Location'] = location

In [37]:
all_alleles.to_csv(
    'MD_genes_exon_variants_Ens_filtered_all_alleles_location.csv',
    header=True, index=False)

In [38]:
#all_alleles = pd.read_csv(
#    '/Users/ksenia/Documents/MODY_genes/pipeline_october2022/MD_genes_exon_variants_Ens_filtered_all_alleles_location.csv', 
#                                converters={'alleles': ast.literal_eval}, low_memory=False) 
all_alleles

,feature_type,consequence_type,clinical_significance,start,end,seq_region_name,assembly_name,alleles,strand,id,source,Gene,Transcript,Exon,vf_allele,Location
0,variation,start_lost,[],57227885.0,57227885.0,3,GRCh38,"[T, A]",1.0,rs1196762591,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686,A,3:57227885
1,variation,missense_variant,[],57227888.0,57227888.0,3,GRCh38,"[C, A]",1.0,rs752732231,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686,A,3:57227888
2,variation,frameshift_variant,[],57227889.0,57227892.0,3,GRCh38,"[GGGG, GGG]",1.0,rs1559504574,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686,GGG,3:57227889
3,variation,missense_variant,[],57227890.0,57227890.0,3,GRCh38,"[G, A]",1.0,rs1022835366,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686,A,3:57227890
4,variation,missense_variant,[],57227891.0,57227891.0,3,GRCh38,"[G, A]",1.0,rs1246196168,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686,A,3:57227891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270659,variation,,,100781275.0,100781275.0,10,GRCh38,"[A, C]",1.0,rs200741999,dbSNP,ENSG00000075891,ENST00000554172,ENSE00002435858,C,10:100781275
270660,variation,,,100781276.0,100781276.0,10,GRCh38,"[G, T]",1.0,rs201925042,dbSNP,ENSG00000075891,ENST00000554172,ENSE00002435858,T,10:100781276
270661,variation,,,100781276.0,100781276.0,10,GRCh38,"[G, C]",1.0,rs201925042,dbSNP,ENSG00000075891,ENST00000554172,ENSE00002435858,C,10:100781276
270662,variation,,,100781340.0,100781340.0,10,GRCh38,"[T, C]",1.0,rs758231319,dbSNP,ENSG00000075891,ENST00000554172,ENSE00002435858,C,10:100781340


In [39]:
#creating the Coordinate column

coordinates = []
for index, row in all_alleles.iterrows():
    coordinates.append(row['Location'] + ':' + row['alleles'][0] + '>' + row['vf_allele'])
all_alleles['coordinate'] = coordinates
all_alleles

,feature_type,consequence_type,clinical_significance,start,end,seq_region_name,assembly_name,alleles,strand,id,source,Gene,Transcript,Exon,vf_allele,Location,coordinate
0,variation,start_lost,[],57227885.0,57227885.0,3,GRCh38,"[T, A]",1.0,rs1196762591,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686,A,3:57227885,3:57227885:T>A
1,variation,missense_variant,[],57227888.0,57227888.0,3,GRCh38,"[C, A]",1.0,rs752732231,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686,A,3:57227888,3:57227888:C>A
2,variation,frameshift_variant,[],57227889.0,57227892.0,3,GRCh38,"[GGGG, GGG]",1.0,rs1559504574,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686,GGG,3:57227889,3:57227889:GGGG>GGG
3,variation,missense_variant,[],57227890.0,57227890.0,3,GRCh38,"[G, A]",1.0,rs1022835366,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686,A,3:57227890,3:57227890:G>A
4,variation,missense_variant,[],57227891.0,57227891.0,3,GRCh38,"[G, A]",1.0,rs1246196168,dbSNP,ENSG00000157500,ENST00000650354,ENSE00003837686,A,3:57227891,3:57227891:G>A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270659,variation,,,100781275.0,100781275.0,10,GRCh38,"[A, C]",1.0,rs200741999,dbSNP,ENSG00000075891,ENST00000554172,ENSE00002435858,C,10:100781275,10:100781275:A>C
270660,variation,,,100781276.0,100781276.0,10,GRCh38,"[G, T]",1.0,rs201925042,dbSNP,ENSG00000075891,ENST00000554172,ENSE00002435858,T,10:100781276,10:100781276:G>T
270661,variation,,,100781276.0,100781276.0,10,GRCh38,"[G, C]",1.0,rs201925042,dbSNP,ENSG00000075891,ENST00000554172,ENSE00002435858,C,10:100781276,10:100781276:G>C
270662,variation,,,100781340.0,100781340.0,10,GRCh38,"[T, C]",1.0,rs758231319,dbSNP,ENSG00000075891,ENST00000554172,ENSE00002435858,C,10:100781340,10:100781340:T>C


In [40]:
all_alleles.to_csv(
    'Ens_filtered_all_alleles_location_coord.csv',
    header=True, index=False)

In [41]:
# To be able to drop duplicates, dropping the 'clinical_significance', 
#'clinical_significance', 'alleles' and other columns

all_alleles_no_duplicates = all_alleles[['id', 'seq_region_name', 'start', 'end', 'strand', 'vf_allele', 'Location', 
             'coordinate', 'Gene', 'Transcript', 'Exon']].drop_duplicates().reset_index(drop=True)
all_alleles_no_duplicates

,id,seq_region_name,start,end,strand,vf_allele,Location,coordinate,Gene,Transcript,Exon
0,rs1196762591,3,57227885.0,57227885.0,1.0,A,3:57227885,3:57227885:T>A,ENSG00000157500,ENST00000650354,ENSE00003837686
1,rs752732231,3,57227888.0,57227888.0,1.0,A,3:57227888,3:57227888:C>A,ENSG00000157500,ENST00000650354,ENSE00003837686
2,rs1559504574,3,57227889.0,57227892.0,1.0,GGG,3:57227889,3:57227889:GGGG>GGG,ENSG00000157500,ENST00000650354,ENSE00003837686
3,rs1022835366,3,57227890.0,57227890.0,1.0,A,3:57227890,3:57227890:G>A,ENSG00000157500,ENST00000650354,ENSE00003837686
4,rs1246196168,3,57227891.0,57227891.0,1.0,A,3:57227891,3:57227891:G>A,ENSG00000157500,ENST00000650354,ENSE00003837686
...,...,...,...,...,...,...,...,...,...,...,...
236708,rs759356936,10,100749850.0,100749850.0,1.0,T,10:100749850,10:100749850:C>T,ENSG00000075891,ENST00000554172,ENSE00002433876
236709,rs758091898,10,100781255.0,100781255.0,1.0,T,10:100781255,10:100781255:C>T,ENSG00000075891,ENST00000554172,ENSE00002435858
236710,rs200741999,10,100781275.0,100781275.0,1.0,G,10:100781275,10:100781275:A>G,ENSG00000075891,ENST00000554172,ENSE00002435858
236711,rs201925042,10,100781276.0,100781276.0,1.0,T,10:100781276,10:100781276:G>T,ENSG00000075891,ENST00000554172,ENSE00002435858


In [44]:
all_alleles_no_duplicates.to_csv(
    'Ens_filtered_all_alleles_location_coord_no_duplicates.csv',
    header=True, index=False)